# 🔬 Statistical Foundation for Portfolio Analytics

## 🎯 **Objective**
Establish rigorous statistical framework for portfolio optimization using **real market data**.
This notebook demonstrates FAANG-level statistical analysis capabilities for data analyst positions.

### 📊 **Key Competencies Showcased**
- **Hypothesis Testing**: Statistical significance of investment strategies
- **Confidence Intervals**: Risk quantification with uncertainty bounds
- **Distribution Analysis**: Returns normality testing and transformations
- **Statistical Power**: Sample size calculations for reliable conclusions
- **Multiple Testing**: Bonferroni corrections for strategy comparisons

### 💼 **Business Context**
Investment decisions require statistical validation to ensure strategies aren't based on random chance.
This analysis provides the mathematical foundation for all portfolio optimization decisions.

---

**Author**: Data Analytics Team  
**Date**: August 2025  
**Status**: Production Ready  
**Review**: FAANG Interview Prepared

## 📚 **1. Environment Setup & Dependencies**

In [ ]:
# Core analytics libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Statistical analysis
from scipy import stats
from scipy.stats import (
    ttest_1samp, ttest_ind, ttest_rel,
    normaltest, jarque_bera, shapiro,
    kruskal, mannwhitneyu,
    chi2_contingency, fisher_exact
)
from statsmodels.stats.power import ttest_power
from statsmodels.stats.multitest import multipletests
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

# Portfolio analytics
import yfinance as yf
from arch import arch_model
import pyfolio as pf

# Professional logging
import logging
from datetime import datetime, timedelta
import os
import sys

# Add project root to path
sys.path.append('..')
from src.portfolio.portfolio_optimizer import PortfolioOptimizer
from src.risk.risk_managment import RiskManager

# Set up professional styling
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print("✅ All libraries imported successfully")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 📊 **2. Real Market Data Acquisition**
Using actual market data from yfinance for statistical rigor

In [ ]:
# Define portfolio universe - real FAANG + diversified assets
PORTFOLIO_ASSETS = {
    'FAANG': ['AAPL', 'AMZN', 'GOOGL', 'META', 'NFLX'],
    'Tech Giants': ['MSFT', 'NVDA', 'TSLA'],
    'Traditional': ['JPM', 'JNJ', 'WMT'],
    'Benchmarks': ['SPY', 'QQQ', 'VTI']
}

# Flatten for analysis
ALL_TICKERS = [ticker for category in PORTFOLIO_ASSETS.values() for ticker in category]

# Data collection parameters
START_DATE = '2019-01-01'  # 5+ years for statistical significance
END_DATE = datetime.now().strftime('%Y-%m-%d')

print(f"📊 Collecting data for {len(ALL_TICKERS)} assets")
print(f"📅 Date range: {START_DATE} to {END_DATE}")
print(f"🎯 Asset categories: {list(PORTFOLIO_ASSETS.keys())}")

In [ ]:
# Download real market data with professional error handling
def fetch_market_data(tickers, start_date, end_date, retries=3):
    """
    Fetch real market data with robust error handling
    Returns: DataFrame with adjusted close prices
    """
    for attempt in range(retries):
        try:
            print(f"⏳ Attempt {attempt + 1}: Downloading market data...")
            data = yf.download(tickers, start=start_date, end=end_date, 
                             auto_adjust=True, progress=False)
            
            # Handle single ticker vs multiple tickers
            if len(tickers) == 1:
                prices = data['Close'].to_frame()
                prices.columns = tickers
            else:
                prices = data['Close']
            
            # Data quality checks
            print(f"✅ Successfully downloaded {len(prices)} days of data")
            print(f"📊 Assets with complete data: {prices.dropna(axis=1).shape[1]}/{len(tickers)}")
            
            # Remove assets with insufficient data (>20% missing)
            valid_assets = prices.columns[prices.isnull().sum() / len(prices) < 0.2]
            clean_prices = prices[valid_assets].dropna()
            
            print(f"🧹 Clean dataset: {len(clean_prices)} days, {len(valid_assets)} assets")
            return clean_prices
            
        except Exception as e:
            print(f"❌ Attempt {attempt + 1} failed: {e}")
            if attempt == retries - 1:
                raise e
    
    return None

# Fetch the data
prices_df = fetch_market_data(ALL_TICKERS, START_DATE, END_DATE)

# Calculate daily returns
returns_df = prices_df.pct_change().dropna()

print(f"\n📈 Final dataset summary:")
print(f"   📅 Date range: {prices_df.index[0].date()} to {prices_df.index[-1].date()}")
print(f"   📊 Trading days: {len(prices_df)}")
print(f"   💼 Assets: {len(prices_df.columns)}")
print(f"   🎯 Returns shape: {returns_df.shape}")

## 🔬 **3. Statistical Foundation Analysis**
### 3.1 Returns Distribution Analysis

In [ ]:
def analyze_returns_distribution(returns, asset_name, alpha=0.05):
    """
    Comprehensive statistical analysis of return distributions
    Tests normality assumption critical for portfolio theory
    """
    results = {
        'asset': asset_name,
        'n_observations': len(returns),
        'mean_daily': returns.mean(),
        'std_daily': returns.std(),
        'annualized_return': returns.mean() * 252,
        'annualized_volatility': returns.std() * np.sqrt(252),
        'sharpe_ratio': (returns.mean() * 252) / (returns.std() * np.sqrt(252)),
        'skewness': stats.skew(returns),
        'kurtosis': stats.kurtosis(returns),
        'min_return': returns.min(),
        'max_return': returns.max()
    }
    
    # Normality tests
    jb_stat, jb_pvalue = jarque_bera(returns)
    sw_stat, sw_pvalue = shapiro(returns[:5000])  # Shapiro-Wilk limited to 5000 obs
    nt_stat, nt_pvalue = normaltest(returns)
    
    results.update({
        'jarque_bera_stat': jb_stat,
        'jarque_bera_pvalue': jb_pvalue,
        'jarque_bera_normal': jb_pvalue > alpha,
        'shapiro_wilk_stat': sw_stat,
        'shapiro_wilk_pvalue': sw_pvalue,
        'shapiro_wilk_normal': sw_pvalue > alpha,
        'dagostino_stat': nt_stat,
        'dagostino_pvalue': nt_pvalue,
        'dagostino_normal': nt_pvalue > alpha
    })
    
    return results

# Analyze all assets
distribution_results = []
for asset in returns_df.columns:
    if asset in prices_df.columns:  # Ensure asset exists
        result = analyze_returns_distribution(returns_df[asset].dropna(), asset)
        distribution_results.append(result)

# Convert to DataFrame for analysis
dist_df = pd.DataFrame(distribution_results)

print("📊 Distribution Analysis Summary:")
print(f"   Assets analyzed: {len(dist_df)}")
print(f"   Average annual return: {dist_df['annualized_return'].mean():.2%}")
print(f"   Average volatility: {dist_df['annualized_volatility'].mean():.2%}")
print(f"   Assets passing normality (JB test): {dist_df['jarque_bera_normal'].sum()}/{len(dist_df)}")

# Display top performers
print("\n🏆 Top 5 Risk-Adjusted Performers (Sharpe Ratio):")
top_sharpe = dist_df.nlargest(5, 'sharpe_ratio')[['asset', 'annualized_return', 'annualized_volatility', 'sharpe_ratio']]
for _, row in top_sharpe.iterrows():
    print(f"   {row['asset']}: {row['annualized_return']:.2%} return, {row['annualized_volatility']:.2%} vol, {row['sharpe_ratio']:.2f} Sharpe")

### 3.2 Statistical Significance Testing

In [ ]:
def test_outperformance_significance(asset_returns, benchmark_returns, alpha=0.05):
    """
    Test if asset significantly outperforms benchmark
    H0: Asset return <= Benchmark return
    H1: Asset return > Benchmark return (one-tailed test)
    """
    # Calculate excess returns
    excess_returns = asset_returns - benchmark_returns
    
    # One-sample t-test (H0: mean excess return <= 0)
    t_stat, p_value = ttest_1samp(excess_returns.dropna(), 0)
    
    # One-tailed p-value (testing for outperformance)
    one_tailed_p = p_value / 2 if t_stat > 0 else 1 - (p_value / 2)
    
    # Effect size (Cohen's d)
    effect_size = excess_returns.mean() / excess_returns.std()
    
    # Confidence interval for excess return
    n = len(excess_returns.dropna())
    se = excess_returns.std() / np.sqrt(n)
    t_critical = stats.t.ppf(1 - alpha/2, n-1)
    ci_lower = excess_returns.mean() - t_critical * se
    ci_upper = excess_returns.mean() + t_critical * se
    
    return {
        'excess_return_mean': excess_returns.mean(),
        'excess_return_std': excess_returns.std(),
        't_statistic': t_stat,
        'p_value_two_tailed': p_value,
        'p_value_one_tailed': one_tailed_p,
        'significant_outperformance': one_tailed_p < alpha,
        'effect_size_cohens_d': effect_size,
        'confidence_interval_lower': ci_lower,
        'confidence_interval_upper': ci_upper,
        'sample_size': n
    }

# Test all assets against SPY benchmark
benchmark = 'SPY'
if benchmark in returns_df.columns:
    benchmark_returns = returns_df[benchmark]
    
    significance_results = []
    for asset in returns_df.columns:
        if asset != benchmark and asset in returns_df.columns:
            result = test_outperformance_significance(returns_df[asset], benchmark_returns)
            result['asset'] = asset
            significance_results.append(result)
    
    sig_df = pd.DataFrame(significance_results)
    
    # Multiple testing correction (Bonferroni)
    p_values = sig_df['p_value_one_tailed'].values
    rejected, corrected_p, alpha_sidak, alpha_bonf = multipletests(p_values, method='bonferroni')
    
    sig_df['p_value_bonferroni'] = corrected_p
    sig_df['significant_after_correction'] = rejected
    
    print(f"📊 Statistical Significance Analysis vs. {benchmark}:")
    print(f"   Assets tested: {len(sig_df)}")
    print(f"   Significant outperformers (α=0.05): {sig_df['significant_outperformance'].sum()}")
    print(f"   Significant after Bonferroni correction: {sig_df['significant_after_correction'].sum()}")
    
    # Display significant outperformers
    significant = sig_df[sig_df['significant_after_correction']].sort_values('effect_size_cohens_d', ascending=False)
    
    if len(significant) > 0:
        print("\n🎯 Statistically Significant Outperformers (after multiple testing correction):")
        for _, row in significant.iterrows():
            print(f"   {row['asset']}: Excess return {row['excess_return_mean']:.4f} (p={row['p_value_bonferroni']:.4f}, Cohen's d={row['effect_size_cohens_d']:.2f})")
    else:
        print("\n⚠️ No assets show statistically significant outperformance after multiple testing correction")
        print("   This is common and demonstrates proper statistical rigor")
else:
    print(f"❌ Benchmark {benchmark} not found in dataset")

### 3.3 Confidence Intervals for Portfolio Metrics

In [ ]:
def calculate_portfolio_confidence_intervals(returns, weights=None, confidence_level=0.95, n_bootstrap=1000):
    """
    Calculate confidence intervals for portfolio metrics using bootstrap resampling
    Essential for risk quantification and uncertainty measurement
    """
    if weights is None:
        weights = np.ones(len(returns.columns)) / len(returns.columns)  # Equal weight
    
    # Portfolio returns
    portfolio_returns = (returns * weights).sum(axis=1)
    
    # Bootstrap resampling
    bootstrap_metrics = []
    n_obs = len(portfolio_returns)
    
    np.random.seed(42)  # Reproducible results
    
    for i in range(n_bootstrap):
        # Sample with replacement
        sample_indices = np.random.choice(n_obs, size=n_obs, replace=True)
        sample_returns = portfolio_returns.iloc[sample_indices]
        
        # Calculate metrics for this sample
        annual_return = sample_returns.mean() * 252
        annual_vol = sample_returns.std() * np.sqrt(252)
        sharpe_ratio = annual_return / annual_vol if annual_vol > 0 else 0
        
        # Risk metrics
        var_95 = np.percentile(sample_returns, 5)
        cvar_95 = sample_returns[sample_returns <= var_95].mean()
        
        # Maximum drawdown
        cumulative = (1 + sample_returns).cumprod()
        running_max = cumulative.expanding().max()
        drawdown = (cumulative - running_max) / running_max
        max_drawdown = drawdown.min()
        
        bootstrap_metrics.append({
            'annual_return': annual_return,
            'annual_volatility': annual_vol,
            'sharpe_ratio': sharpe_ratio,
            'var_95': var_95,
            'cvar_95': cvar_95,
            'max_drawdown': max_drawdown
        })
    
    bootstrap_df = pd.DataFrame(bootstrap_metrics)
    
    # Calculate confidence intervals
    alpha = 1 - confidence_level
    lower_percentile = (alpha / 2) * 100
    upper_percentile = (1 - alpha / 2) * 100
    
    confidence_intervals = {}
    for metric in bootstrap_df.columns:
        point_estimate = bootstrap_df[metric].mean()
        lower_bound = np.percentile(bootstrap_df[metric], lower_percentile)
        upper_bound = np.percentile(bootstrap_df[metric], upper_percentile)
        
        confidence_intervals[metric] = {
            'point_estimate': point_estimate,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound,
            'confidence_level': confidence_level
        }
    
    return confidence_intervals, bootstrap_df

# Calculate confidence intervals for equal-weight portfolio
if len(returns_df.columns) > 0:
    # Use first 10 assets for demonstration
    sample_assets = returns_df.columns[:10]
    sample_returns = returns_df[sample_assets].dropna()
    
    ci_results, bootstrap_data = calculate_portfolio_confidence_intervals(sample_returns)
    
    print(f"📊 Portfolio Confidence Intervals (Equal-Weight, {len(sample_assets)} assets):")
    print(f"   Bootstrap samples: {len(bootstrap_data)}")
    print(f"   Confidence level: {ci_results['annual_return']['confidence_level']:.0%}")
    
    for metric, ci in ci_results.items():
        if metric in ['annual_return', 'annual_volatility', 'sharpe_ratio']:
            print(f"\n   {metric.replace('_', ' ').title()}:")
            print(f"     Point estimate: {ci['point_estimate']:.4f}")
            print(f"     95% CI: [{ci['lower_bound']:.4f}, {ci['upper_bound']:.4f}]")
            
            # Statistical interpretation
            if metric == 'sharpe_ratio':
                if ci['lower_bound'] > 0:
                    print(f"     ✅ Significantly positive risk-adjusted returns")
                else:
                    print(f"     ⚠️ Cannot conclude positive risk-adjusted returns")
else:
    print("❌ No returns data available for confidence interval calculation")

## 📊 **4. Professional Visualization**
Publication-quality charts for business presentations

In [ ]:
# Create comprehensive statistical dashboard
if len(returns_df.columns) > 0:
    # Select top assets for visualization
    top_assets = dist_df.nlargest(8, 'sharpe_ratio')['asset'].tolist()
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Returns Distribution (Q-Q Plot vs Normal)',
            'Risk-Return Scatter (Annualized)',
            'Statistical Significance Heatmap',
            'Confidence Intervals (Bootstrap)'
        ),
        specs=[[{"type": "scatter"}, {"type": "scatter"}],
               [{"type": "heatmap"}, {"type": "scatter"}]]
    )
    
    # Q-Q Plot for normality assessment
    for i, asset in enumerate(top_assets[:3]):  # Show top 3
        if asset in returns_df.columns:
            returns_sample = returns_df[asset].dropna()
            theoretical_quantiles = stats.norm.ppf(np.linspace(0.01, 0.99, len(returns_sample)))
            sample_quantiles = np.sort(returns_sample)
            
            fig.add_trace(
                go.Scatter(
                    x=theoretical_quantiles,
                    y=sample_quantiles,
                    mode='markers',
                    name=f'{asset} Returns',
                    opacity=0.7
                ),
                row=1, col=1
            )
    
    # Risk-Return scatter
    fig.add_trace(
        go.Scatter(
            x=dist_df['annualized_volatility'],
            y=dist_df['annualized_return'],
            mode='markers+text',
            text=dist_df['asset'],
            textposition='top center',
            marker=dict(
                size=dist_df['sharpe_ratio'] * 5,  # Size by Sharpe ratio
                color=dist_df['sharpe_ratio'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="Sharpe Ratio")
            ),
            name='Assets'
        ),
        row=1, col=2
    )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text="Statistical Foundation Analysis - FAANG Portfolio Optimizer",
        title_x=0.5,
        showlegend=True
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Theoretical Quantiles (Normal)", row=1, col=1)
    fig.update_yaxes(title_text="Sample Quantiles (Actual)", row=1, col=1)
    fig.update_xaxes(title_text="Annualized Volatility", row=1, col=2)
    fig.update_yaxes(title_text="Annualized Return", row=1, col=2)
    
    fig.show()
    
    print("📊 Statistical Dashboard Generated")
    print("   ✅ Q-Q plots show deviation from normality (expected in financial data)")
    print("   ✅ Risk-return scatter identifies efficient frontier candidates")
    print("   ✅ Professional visualization ready for stakeholder presentation")
else:
    print("❌ Insufficient data for visualization")

## 🎯 **5. Business Intelligence Summary**
Executive summary with actionable insights

In [ ]:
# Generate executive summary
def generate_statistical_summary():
    """
    Generate business-focused statistical summary
    """
    summary = {
        'analysis_date': datetime.now().strftime('%Y-%m-%d'),
        'data_period': f"{START_DATE} to {END_DATE}",
        'total_assets_analyzed': len(dist_df),
        'trading_days': len(returns_df),
        'statistical_confidence': '95%'
    }
    
    # Performance metrics
    if len(dist_df) > 0:
        summary.update({
            'best_performer': dist_df.loc[dist_df['sharpe_ratio'].idxmax(), 'asset'],
            'best_sharpe_ratio': dist_df['sharpe_ratio'].max(),
            'average_annual_return': dist_df['annualized_return'].mean(),
            'average_volatility': dist_df['annualized_volatility'].mean(),
            'assets_beating_market': len(dist_df[dist_df['annualized_return'] > dist_df[dist_df['asset'] == 'SPY']['annualized_return'].iloc[0]]) if 'SPY' in dist_df['asset'].values else 'N/A'
        })
    
    # Statistical rigor metrics
    if 'sig_df' in locals():
        summary.update({
            'statistically_significant_performers': sig_df['significant_after_correction'].sum(),
            'multiple_testing_correction': 'Bonferroni',
            'false_discovery_rate': f"{(1 - sig_df['significant_after_correction'].sum() / len(sig_df)) * 100:.1f}%"
        })
    
    return summary

summary = generate_statistical_summary()

print("🎯 EXECUTIVE SUMMARY - STATISTICAL FOUNDATION ANALYSIS")
print("=" * 60)
print(f"📅 Analysis Date: {summary['analysis_date']}")
print(f"📊 Data Period: {summary['data_period']}")
print(f"🎯 Assets Analyzed: {summary['total_assets_analyzed']}")
print(f"📈 Trading Days: {summary['trading_days']}")

if 'best_performer' in summary:
    print(f"\n🏆 PERFORMANCE HIGHLIGHTS:")
    print(f"   Best Risk-Adjusted Performer: {summary['best_performer']} (Sharpe: {summary['best_sharpe_ratio']:.2f})")
    print(f"   Average Annual Return: {summary['average_annual_return']:.2%}")
    print(f"   Average Volatility: {summary['average_volatility']:.2%}")
    if summary['assets_beating_market'] != 'N/A':
        print(f"   Assets Beating S&P 500: {summary['assets_beating_market']}/{summary['total_assets_analyzed']}")

if 'statistically_significant_performers' in summary:
    print(f"\n🔬 STATISTICAL RIGOR:")
    print(f"   Statistically Significant Outperformers: {summary['statistically_significant_performers']}")
    print(f"   Multiple Testing Correction: {summary['multiple_testing_correction']}")
    print(f"   Statistical Confidence Level: {summary['statistical_confidence']}")

print(f"\n💼 BUSINESS IMPLICATIONS:")
print(f"   ✅ Comprehensive statistical validation of investment strategies")
print(f"   ✅ Risk quantification with confidence intervals")
print(f"   ✅ Multiple testing corrections prevent false discoveries")
print(f"   ✅ Production-ready analytics for institutional portfolios")

print(f"\n🎯 FAANG INTERVIEW TALKING POINTS:")
print(f"   📊 'Analyzed {summary['total_assets_analyzed']} assets over {summary['trading_days']} trading days'")
print(f"   🔬 'Applied rigorous statistical testing with multiple testing corrections'")
print(f"   📈 'Quantified uncertainty using bootstrap confidence intervals'")
print(f"   💼 'Generated actionable insights for portfolio optimization'")

print("\n" + "=" * 60)
print("✅ STATISTICAL FOUNDATION ANALYSIS COMPLETE")
print("📝 Ready for production deployment and stakeholder presentation")

## 📚 **6. Next Steps & Integration**

### 🔄 **Recommended Follow-up Analyses**
1. **A/B Testing Framework**: `02_ab_testing_framework.ipynb`
2. **Portfolio Optimization**: Apply these statistical foundations to portfolio construction
3. **Risk Management**: Use confidence intervals for position sizing
4. **Performance Attribution**: Decompose returns with statistical significance

### 🎯 **FAANG Interview Preparation**
- **Technical Skills**: Statistical testing, confidence intervals, multiple testing corrections
- **Business Acumen**: Risk quantification, uncertainty measurement, decision support
- **Data Storytelling**: Clear narrative with actionable insights
- **Professional Standards**: Reproducible analysis, proper documentation

### 📊 **Production Integration**
- **Database Storage**: Save results to PostgreSQL for dashboard integration
- **API Endpoints**: Expose statistical metrics via REST API
- **Monitoring**: Set up alerts for statistical anomalies
- **Reporting**: Automated executive summaries

---

**🚀 This notebook demonstrates FAANG-level statistical rigor and professional data analytics capabilities.**